<a href="https://colab.research.google.com/github/saipra003/locust-gene-project/blob/master/GenBank_Annotations_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# install any required dependencies for program to run
!pip install biopython

In [10]:
# import all the required libraries
from Bio import SeqIO
import pandas as pd
from datetime import datetime as dt
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import os
import re 

In [11]:
from google.colab import drive # mount Google Drive onto the Colab platform
drive.mount("/content/gdrive/", force_remount = True)
! mkdir "/content/gdrive/My Drive/Colab Notebooks/flybase_files"
path = "/content/gdrive/My Drive/Colab Notebooks/flybase_files" # change the working directory to the one that has all the flybase files in it
os.chdir(path)

Mounted at /content/gdrive/
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
mkdir: cannot create directory ‘/content/gdrive/My Drive/Colab Notebooks/flybase_files’: File exists


In [12]:
from Bio import SeqIO
record = SeqIO.read("para.gb", "genbank")
print(record)

ID: chromosome:BDGP6.28:X:16453027:16535096:1
Name: X
Description: Drosophila melanogaster chromosome X BDGP6.28 partial sequence 16453027..16535096 reannotated via EnsEMBL
Number of features: 595
/data_file_division=HTG
/date=25-NOV-2020
/accessions=['chromosome:BDGP6.28:X:16453027:16535096:1']
/keywords=['']
/source=fruit fly
/organism=Drosophila melanogaster
/taxonomy=['Eukaryota', 'Opisthokonta', 'Metazoa', 'Eumetazoa', 'Bilateria', 'Protostomia', 'Ecdysozoa', 'Panarthropoda', 'Arthropoda', 'Mandibulata', 'Pancrustacea', 'Hexapoda', 'Insecta', 'Dicondylia', 'Pterygota', 'Neoptera', 'Holometabola', 'Diptera', 'Brachycera', 'Muscomorpha', 'Eremoneura', 'Cyclorrhapha', 'Schizophora', 'Acalyptratae', 'Ephydroidea', 'Drosophilidae', 'Drosophilinae', 'Drosophilini']
/comment=This sequence was annotated by Ensembl. Please visit the Ensembl or EnsemblGenomes
web site, http://www.ensembl.org/ or http://www.ensemblgenomes.org/ for more
information.
All feature locations are relative to the f

In [13]:
work_features = []
print(len(work_features))
for feature in record.features:
  if ((feature.type == 'mRNA') or (feature.type == 'CDS')):
    work_features.append(feature)
print(len(work_features))

0
132


In [14]:
utr_df = pd.DataFrame()

five_prime_utr_start_list = []
three_prime_utr_start_list = []
five_prime_utr_end_list = []
three_prime_utr_end_list = []


for feature in work_features:
  if (feature.type == 'mRNA'):    
    five_prime_utr_start = int(feature.location.start)
    three_prime_utr_end = int(feature.location.end)
    five_prime_utr_start_list.append(five_prime_utr_start)
    three_prime_utr_end_list.append(three_prime_utr_end)
  elif (feature.type == 'CDS'):
    five_prime_utr_end = int(feature.location.start) - 1
    three_prime_utr_start = int(feature.location.end) + 1
    three_prime_utr_start_list.append(three_prime_utr_start)
    five_prime_utr_end_list.append(five_prime_utr_end)

utr_df['five_prime_utr_start'] =  five_prime_utr_start_list
utr_df['five_prime_utr_end'] =  five_prime_utr_end_list
utr_df['three_prime_utr_start'] =  three_prime_utr_start_list
utr_df['three_prime_utr_end'] =  three_prime_utr_end_list

utr_df

,five_prime_utr_start,five_prime_utr_end,three_prime_utr_start,three_prime_utr_end
0,16448664,16448696,16452086,16453615
1,16449151,16449482,16452086,16453615
2,16448664,16448696,16452086,16454972
3,16448664,16448696,16452086,16454972
4,16448664,16448696,16452086,16453615
...,...,...,...,...
61,10576,10760,68747,74388
62,10576,10760,68747,74388
63,10576,33806,68747,74388
64,10576,10760,68747,74388


In [15]:
# 1. Define the start location and end location for the feature using a for loop through the pandas dataframe

counter=0
cols = list(utr_df.columns)
final_features = []
from Bio import SeqFeature
from Bio.SeqFeature import FeatureLocation
from Bio.SeqFeature import SeqFeature
for ind in utr_df.index:
  from Bio import SeqFeature
  five_prime_start = SeqFeature.ExactPosition((utr_df[cols[0]][ind]) - 1)
  five_prime_end = SeqFeature.ExactPosition(utr_df[cols[1]][ind])
  three_prime_start = SeqFeature.ExactPosition(utr_df[cols[2]][ind] - 1)
  three_prime_end = SeqFeature.ExactPosition(utr_df[cols[3]][ind])
  # define the locations for the UTRs
  from Bio.SeqFeature import FeatureLocation
  from Bio.SeqFeature import SeqFeature
  three_prime_utr = SeqFeature(FeatureLocation(three_prime_start, three_prime_end), type = '3\'UTR')
  five_prime_utr = SeqFeature(FeatureLocation(five_prime_start, five_prime_end), type = '5\'UTR')
  final_features.append(three_prime_utr)
  final_features.append(five_prime_utr)
  counter+=1

  
  # remove the mRNA features from the working_list
counter = 1

for rec in work_features:
  if (rec.type == 'CDS'):
    final_features.append(rec)
    # counter += 1

# print(len(final_features))
final_features

[SeqFeature(FeatureLocation(ExactPosition(16452085), ExactPosition(16453615)), type="3'UTR"),
 SeqFeature(FeatureLocation(ExactPosition(16448663), ExactPosition(16448696)), type="5'UTR"),
 SeqFeature(FeatureLocation(ExactPosition(16452085), ExactPosition(16453615)), type="3'UTR"),
 SeqFeature(FeatureLocation(ExactPosition(16449150), ExactPosition(16449482)), type="5'UTR"),
 SeqFeature(FeatureLocation(ExactPosition(16452085), ExactPosition(16454972)), type="3'UTR"),
 SeqFeature(FeatureLocation(ExactPosition(16448663), ExactPosition(16448696)), type="5'UTR"),
 SeqFeature(FeatureLocation(ExactPosition(16452085), ExactPosition(16454972)), type="3'UTR"),
 SeqFeature(FeatureLocation(ExactPosition(16448663), ExactPosition(16448696)), type="5'UTR"),
 SeqFeature(FeatureLocation(ExactPosition(16452085), ExactPosition(16453615)), type="3'UTR"),
 SeqFeature(FeatureLocation(ExactPosition(16448663), ExactPosition(16448696)), type="5'UTR"),
 SeqFeature(FeatureLocation(ExactPosition(68746), ExactPosit

In [20]:
record.features = final_features
# Save as GenBank file
output_file = open('para_complete_seqFeature.gb', 'w')
SeqIO.write(record, output_file, 'genbank')

1

In [18]:
len(record.seq)

82070